本作業之目的是在建立一文本之TF-IDF模型之後，找出文本中的句子們和新句子的相似度
有些同學們在求出新句子的TFIDF向量後，不知如何比較相似度

1.整理文本(corpus and dictionary)

In [8]:
corpus = [
    'this is the first document',
    'this is the second second document',
    'and the third one',
    'is this the first document'
]
word_list = []
for i in range(len(corpus)):
    word_list.append(corpus[i].split(' '))
print("word list:", word_list)

from gensim import corpora
dictionary = corpora.Dictionary(word_list)
new_corpus = [dictionary.doc2bow(text) for text in word_list]
print("new corpus:", new_corpus)
print("dictionry:", dictionary.token2id)





word list: [['this', 'is', 'the', 'first', 'document'], ['this', 'is', 'the', 'second', 'second', 'document'], ['and', 'the', 'third', 'one'], ['is', 'this', 'the', 'first', 'document']]
new corpus: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(0, 1), (2, 1), (3, 1), (4, 1), (5, 2)], [(3, 1), (6, 1), (7, 1), (8, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]
dictionry: {'document': 0, 'first': 1, 'is': 2, 'the': 3, 'this': 4, 'second': 5, 'and': 6, 'one': 7, 'third': 8}


2.建立此文本的 TFIDF model

In [9]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("my_model.tfidf")

tfidf = models.TfidfModel.load("my_model.tfidf")


3.印出此文本每個句子TF-IDF 向量 (注意：常用字 the 被去除)

In [10]:
tfidf_vec = []
for i in range(len(corpus)):
    string = corpus[i]
    string_bow = dictionary.doc2bow(string.lower().split())
    print (string_bow)
    string_tfidf = tfidf[string_bow]
    print (string_tfidf)
    tfidf_vec.append(string_tfidf)
print(tfidf_vec)



[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
[(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)]
[(0, 1), (2, 1), (3, 1), (4, 1), (5, 2)]
[(0, 0.10212329019650272), (2, 0.10212329019650272), (4, 0.10212329019650272), (5, 0.9842319344536239)]
[(3, 1), (6, 1), (7, 1), (8, 1)]
[(6, 0.5773502691896258), (7, 0.5773502691896258), (8, 0.5773502691896258)]
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
[(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)]
[[(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)], [(0, 0.10212329019650272), (2, 0.10212329019650272), (4, 0.10212329019650272), (5, 0.9842319344536239)], [(6, 0.5773502691896258), (7, 0.5773502691896258), (8, 0.5773502691896258)], [(0, 0.33699829595119235), (1, 0.8119707171924228), (2, 0.33699829595119235), (4, 0.33699829595119235)]]


4.新句子的每個字TF-IDF value及句子向量 (注意：the 消失了，i 和 name 不存在文本中)

In [11]:
string = 'the i first second name'
print (string.lower().split())
string_bow = dictionary.doc2bow(string.lower().split())
print (string_bow)
string_tfidf = tfidf[string_bow]
print(string_tfidf)

['the', 'i', 'first', 'second', 'name']
[(1, 1), (3, 1), (5, 1)]
[(1, 0.4472135954999579), (5, 0.8944271909999159)]


5.和原來的4個句子比較，特徵數等於文本句數

In [12]:
from gensim import similarities

index = similarities.SparseMatrixSimilarity(tfidf[new_corpus],  num_features=len(dictionary))

In [13]:
sims = index[string_tfidf]
print(list(enumerate(sims)))

[(0, 0.36312434), (1, 0.8803238), (2, 0.0), (3, 0.36312434)]


6.相似度排序

In [14]:
for document_number, score in sorted(enumerate(sims), key=lambda x: x[1], reverse=True):
    print(document_number, score)

1 0.8803238
0 0.36312434
3 0.36312434
2 0.0
